Objetivo:
 
Entender o hábito de consumo de fast-food entre os jovens adultos da cidade de São Paulo. Considerado como jovens adultos, pessoas com a faixa etária entre 20 a 34 anos.

____________________________________________________________________________________________________________________________________


Tipo de método amostral utilizado: 

Por Conglomerado.

____________________________________________________________________________________________________________________________________

Justificativa do uso deste método:

1º Menor custo logístico: nesta amostragem a população foi divida em conglomerados (regiões da cidade > subprefeituras > bairros), então foi selecionado aleatoriamente alguns desses conglomerados para este estudo. O que reduz o custo de descolocamento e coleta de dados, por centralizar em áreas específicas.

2º Facilidade de implementar: por ser realizado em áreas específicas, facilita a alocação de pesquisadores, sendo mais eficiente e gerenciável.

3º Representatividade: conglomerados foram bem escolhidos, sendo diversos e representando a variação existente na população (bairros ricos, pobres, urbanos e suburbanos).

____________________________________________________________________________________________________________________________________

Coleta do tamanho da população, com base nas estimativas de população de 2022 do IBGE. 

Fonte: IBGE - Censo Demográfico 
Link: https://sidra.ibge.gov.br/tabela/9514#/n1/all/n6/3550308/v/allxp/p/all/c2/6794/c287/6653,49109,60040,60041,93070,93084,93085,93087,93088,93089,93091,93092,93093,93094,93095,93096,93097,93098,100362/c286/113635/l/v,p+c2+c287,t+c286/resultado


Tamanho da amostra:

População: jovens adultos, consideradas faixa etária entre 20 a 34 anos, da cidade de São Paulo.

Margem de Erro: Entre 3% a 5%

Nível de Confiança: 95%

Proporção de variabilidade: 50% 

In [30]:
import pandas as pd
import requests
import io

# URL com arquivo em XLSX, do IBGE
url = "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela9514.xlsx&terr=N&rank=-&query=t/9514/n1/all/n6/3550308/v/allxp/p/all/c2/6794/c287/6653,49108,49109,60040,60041,93070,93084,93085,93086,93087,93088,93089,93090,93091,93092,93093,93094,93095,93096,93097,93098,100362/c286/113635/l/v,p%2Bc2%2Bc287,t%2Bc286"

# Requisição
response = requests.get(url)

# Verificação de sucesso
if response.status_code == 200:
    # Ler arquivo
    xlsx_content = response.content
    # Criar DataFrame
    df = pd.read_excel(io.BytesIO(xlsx_content), header=None)  # Ignorar o cabeçalho existente

else:
    print("Falha ao acessar o arquivo XLSX.")


In [38]:
# Selecionar as linhas 4 e 6
df_sp = df.iloc[[5, 7]]

# Manter as duas primeiras colunas e transpor as demais
df_sp = df_sp.T

# renomeia colunas
rotulo = ['Faixa Etaria', 'Numero_Habitantes']

# filtra linhas
df_sp = df_sp.iloc[3:]

# aplica rótulos
df_sp.columns = rotulo

# função agrupa idade em categoria
def categorizar_idade(value):
    if value == '20 a 24 anos' or value == '30 a 34 anos':
        return 'jovens_adultos'
    else:
        return 'outros'

df_sp['Categoria'] = df_sp['Faixa Etaria'].apply(lambda x: categorizar_idade(x))

# agrupa por categoria e numero de habitantes
df_agrupado = df_sp.groupby('Categoria').agg({'Numero_Habitantes': 'sum'}).reset_index()

# adiciona percentagem 
total_habitantes = df_agrupado['Numero_Habitantes'].sum()
df_agrupado['% sobre total'] = ((df_agrupado['Numero_Habitantes'] / total_habitantes) * 100).apply(lambda x: round(x, 2))


# Adicionar uma nova linha com o total de número de habitantes
total_row = {'Categoria': 'Total', 'Numero_Habitantes': total_habitantes, '% sobre total': 100}
df_agrupado = pd.concat([df_agrupado, pd.DataFrame(total_row, index=[len(df_agrupado)])], ignore_index=True)

print(df_agrupado)



        Categoria Numero_Habitantes  % sobre total
0  jovens_adultos           1733707          15.14
1          outros           9718292          84.86
2           Total          11451999         100.00


Tamanho da amostra:

População (N): 1,7 milhões de jovens adultos.

Margem de Erro (E): 5%

Nível de Confiança: 95% (Z= 1,96)

Proporção (p): 50% 
Complemento da proporção (q): 50%

In [45]:
import math
import locale

# define localidade para Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

number_jovens_adultos = df_agrupado[df_agrupado['Categoria']=='jovens_adultos']['Numero_Habitantes'].iloc[0]

N = int(number_jovens_adultos)      # população de jovens adultos
Z = 1.96                            # nível de confiança
E = 0.05                            # margem de erro de 5%
p = 0.5                             # proporção
q = 1 - p                           # complemento da proporção

# tamanho da amostra
n = (Z**2 * p * q) / (E**2)

# arredondar para cima
n = math.ceil(n)

print('Parâmetros usados para definição do tamanho da amostra \n')
print(f'População: {locale.format_string("%d", N, grouping=True)} jovens adultos')
print(f'Nível de Confiança: 95%')
print(f'Margem de Erro {E * 100}% \n')
print(f'Tamanho da Amostra: {n}')

Parâmetros usados para definição do tamanho da amostra 

População: 1.733.707 jovens adultos
Nível de Confiança: 95%
Margem de Erro 5.0% 

Tamanho da Amostra: 385
